In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.llms import Ollama

In [27]:
!pip install -qU langchain-ollama

In [29]:
%pip install -U ollama

Note: you may need to restart the kernel to use updated packages.


In [31]:
from langchain_ollama import ChatOllama

In [33]:
llm = ChatOllama(
    model="llama3.2")

In [35]:
llm.invoke(question)

AIMessage(content="DeepSeek is an AI research project that aims to develop a knowledge graph-based model for natural language processing (NLP) tasks. The reasoning capabilities of DeepSeek are based on its ability to process and integrate information from multiple sources, making connections between different concepts and entities.\n\nHere's how DeepSeek develops reasoning capabilities:\n\n1. **Knowledge Graph Representation**: DeepSeek represents knowledge as a graph of entities, relationships, and attributes. This graph is used to store and retrieve information, allowing the model to make inferences and draw conclusions based on the available data.\n2. **Graph Neural Networks (GNNs)**: DeepSeek uses GNNs to process the graph structure and extract relevant information. GNNs are designed to propagate information through the graph, allowing the model to reason about complex relationships between entities.\n3. **Attention Mechanisms**: DeepSeek employs attention mechanisms to focus on sp

In [51]:
question = "What is the deal between US and China on May 11, 2025"

In [53]:
llm.invoke(question)

AIMessage(content="I don't have information about a specific agreement or deal between the US and China on May 11, 2025. As my knowledge cutoff is December 2023, I do not have real-time information or future events.\n\nHowever, I can suggest some possible sources where you may be able to find information about potential agreements or developments between the US and China:\n\n1. Official government websites: You can check the official websites of the US Department of State, the US Department of Commerce, and the Chinese Ministry of Foreign Affairs for press releases and statements related to their interactions.\n2. News outlets: Reputable news organizations such as CNN, BBC, and Reuters often report on developments between the US and China.\n3. Think tanks and research institutions: Organizations like the Brookings Institution, the Council on Foreign Relations, and the Peterson Institute for International Economics may publish analysis and reports on US-China relations.\n\nIf you're loo

In [55]:
# Load the paper
loader = PyMuPDFLoader("U.S. Announces China Trade Deal in Geneva – The White House.pdf")
documents = loader.load()

# Split the documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

# Create embeddings
embeddings = HuggingFaceEmbeddings()

# Create a vector store
db = Chroma.from_documents(texts, embeddings)

In [57]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

retriever = db.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to augment your own knowledge."),
    ("human", "Context: {context}"),
    ("human", "Question: {input}"),
    ("human", "Please provide a detailed answer, combining information from the context (if relevant) and your own knowledge.")
])

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [59]:
retriever.invoke(question)

[Document(metadata={'author': '', 'creationDate': '', 'creationdate': '', 'creator': '', 'file_path': 'Economic_And_Trade_Agreement_Between_The_United_States_And_China_Text.pdf', 'format': 'PDF 1.6', 'keywords': '', 'modDate': '', 'moddate': '', 'page': 0, 'producer': '', 'source': 'Economic_And_Trade_Agreement_Between_The_United_States_And_China_Text.pdf', 'subject': '', 'title': '', 'total_pages': 91, 'trapped': ''}, page_content='ECONOMIC AND TRADE AGREEMENT BETWEEN THE \nGOVERNMENT OF THE UNITED STATES OF AMERICA AND THE \nGOVERNMENT OF THE PEOPLE’S REPUBLIC OF CHINA \n \n  \nPREAMBLE \n \n \nThe Government of the United States of America and the Government of the People’s \nRepublic of China (collectively the “Parties”), \nRECOGNIZING the importance of their bilateral economic and trade relationship;  \nREALIZING that it is in the interests of both countries that trade grow and that there is \nadherence to international norms so as to promote market-based outcomes;   \nCONVINCED o

In [61]:
def ask_question(chain, question):
    result = chain.invoke({"input": question})
    print("Question:", question)
    print("\n ** WITH CONTEXT **\n")
    print("Answer:", result['answer'])
    print("\nSources:")
    for doc in result['context']:
        print(doc.metadata)
    print("\n")

In [63]:

ask_question(rag_chain, question)

Question: What is the deal between US and China on May 11, 2025

 ** WITH CONTEXT **

Answer: Based on the provided context, it appears that the "deal" between the US and China refers to an economic and trade agreement signed by both countries in Geneva, Switzerland on May 11, 2025.

According to the text, Secretary of the Treasury Scott Bessent stated that the talks were productive, and U.S. Trade Representative Ambassador Jamieson Greer described it as a "very constructive" two days. The agreement aims to address existing trade and investment concerns between the two countries.

The main provisions of the agreement are outlined in Chapter 6: Expanding Trade. Article 6.1 sets out four objectives:

1. To increase imports of high-quality goods and services from the US, including manufactured goods, agricultural goods, energy products, and services.
2. To enhance trade flows between the two countries by improving market access and reducing trade barriers.
3. To promote economic restructu